# Tugas Besar II
## Feed Forward Neural Network
### IF4071 Pembelajaran Mesin

### 1.a Create Classifier
 
Membuat kode program Backpropagation (lihat contoh algoritma di buku Tom Mitchell hal. 98), dengan spesifikasi berikut:
* Jumlah hidden layer maksimal 10
* Jumlah node dalam setiap hidden layer dapat bervariasi
* Fully-connected layer
* Fungsi aktivasi berupa sigmoid untuk semua hidden layer maupun output layer
* Node output berjumlah 1
* Program memberikan pilihan untuk menggunakan momentum atau tidak
* Program mengimplementasikan mini-batch stochastic gradient descent
* Implementasi incremental dengan setting batch-size=1 dan implementasi batch dengan setting batch-size=jumlah data.

In [1]:
# import numpy as np
# import random

# class Network(object):
#     def __init__(self, sizes):
#         self.num_layers = len(sizes)
#         self.sizes = sizes
#         self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
#         self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
# NN = Network([1,2,3])
# print(NN.biases)
# print(NN.weights)

[array([[-0.14960468],
       [-1.82972203]]), array([[-0.93521408],
       [ 1.07297066],
       [-1.1133406 ]])]
[array([[-0.28690104],
       [-0.61859377]]), array([[ 0.29720633, -1.0068917 ],
       [-0.52448592, -0.60944934],
       [ 0.66115332,  1.01335705]])]


In [93]:
"""
network.py
~~~~~~~~~~
A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
            # feedforward
        activation = a
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b.transpose()[0]
            activation = sigmoid(z)
        return activation

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
#             random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch " +  str(j) + ": ")
                print(self.evaluate(test_data))
                print("/" +  str(n_test))
            else:
                print("Epoch " + str(j) + " complete")

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        print("FORWARD")
        # feedforward
        activation = x
        print(activation)
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b.transpose()[0]
            print(z)
            zs.append(z)
            activation = sigmoid(z)
            print(activation)
            activations.append(activation)
        
        print("BACKWARD")
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        print("delta")
        print(delta)
        print("activation")
#         print(activations[-2].shape)
        print(activations[-2])
        print(activations[-2][0] * delta)
        nabla_w[-1] = multiply(delta, activations[-2])
        print("nabla_w[-1]")
        print(nabla_w[-1])
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            print("delta")
            print(delta)
            print("activation")
    #         print(activations[-2].shape)
            print(activations[-l-1])
            print("activation trans")
            print(activations[-l-1].transpose())
            nabla_w[-l] = multiply(delta, activations[-l-1].transpose())
            print("nabla_w[-l]")
            print(nabla_w[-l])
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(self.feedforward(x), y)
                        for (x, y) in test_data]
        print(test_results)
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return np.squeeze(output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

def multiply(A, B):
    result = []
    for i in range(len(A)) :
        row = []
        for j in range(len(B)):
            row.append(A[i]*B[j])
        result.append(row)
        
    return row
    

In [21]:
a = np.array([i for i in range(20)])
# a = np.zeros((10, 2))
# a = a.reshape(2,-1)
print(a)
print(a.shape)
print(np.multiply(a,2))
b = a.T
print(b.shape)

print(np.dot(a,b))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
(20,)
[ 0  2  4  6  8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38]
(20,)
2470


In [94]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import shutil

import numpy as np

from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
# from tensorflow.contrib import skflow

iris = datasets.load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data[0:100], iris.target[0:100],
    test_size=0.2, random_state=42)
train_data = [(x, y) for x, y in zip(X_train, y_train)]
test_data = [(x, y) for x, y in zip(X_test, y_test)]

# new_classifier = skflow.TensorFlowEstimator.restore('/home/ubuntu/scratch/skflow_batch/batch_model_2016_04_23')
neural_network = Network([4,3,2,1])
# neural_network = Network([2,2,1])
# neural_network.biases[0][0][0] = 0.1
# neural_network.biases[0][1][0] = 0.1
# neural_network.biases[1][0][0] = 0.2
# print(neural_network.biases)
# neural_network.weights[1][0][0] = 0.2
# neural_network.weights[1][0][1] = 0.3
# neural_network.weights[0][0][0] = -0.2
# neural_network.weights[0][0][1] = 0.1
# neural_network.weights[0][1][0] = -0.1
# neural_network.weights[0][1][1] = 0.3
# print("WEIGHTS")
# print(neural_network.weights)
# print(train_data)
# neural_network.SGD([(np.array([0.1, 0.9]), 0.9)], 1, 1, 0.001)
neural_network.SGD(train_data, 1, 3, 0.001, test_data)

# check model load with test data
# score = metrics.accuracy_score(y_test, new_classifier.predict(X_test))
# print('Accuracy: {0:f}'.format(score))

WEIGHTS
[array([[-1.05842085,  2.30908699, -2.85606944, -0.60904877],
       [-0.12297407, -0.22636714, -0.39407608, -0.03515335],
       [ 0.5051451 ,  0.43217198,  0.3226022 ,  1.1905234 ]]), array([[-0.51815453,  1.1340711 , -2.08558534],
       [-0.32403517, -0.57574538, -0.02384421]]), array([[-0.3503622 ,  0.19502961]])]
FORWARD
[5.7 2.8 4.5 1.3]
[-13.23240573  -0.66821955   8.71618066]
[1.79158918e-06 3.38895629e-01 9.99836115e-01]
[-1.32218674  0.63656907]
[0.21045471 0.65397748]
[1.05320212]
[0.74138932]
BACKWARD
delta
[-0.04958374]
activation
[0.21045471 0.65397748]
[-0.01043513]
nabla_w[-1]
[-0.010435130420976869, -0.03242664623497391]
delta
[ 0.00288664 -0.0021883 ]
activation
[1.79158918e-06 3.38895629e-01 9.99836115e-01]
activation trans
[1.79158918e-06 3.38895629e-01 9.99836115e-01]
nabla_w[-l]
[-3.920535313357591e-09, -0.0007416054399598038, -0.0021879417689428817]
delta
[-1.40932913e-09  1.01572216e-03 -9.77930898e-07]
activation
[5.7 2.8 4.5 1.3]
activation trans
[5.7

[-1.4146850268517737e-09, -0.0006515236563865312, -0.0021619310923829695]
delta
[-4.94208012e-10  9.32942918e-04 -5.60254235e-07]
activation
[6.6 2.9 4.6 1.3]
activation trans
[6.6 2.9 4.6 1.3]
nabla_w[-l]
[-3.6976779527930594e-06, -1.6247372822878595e-06, -2.577169482249708e-06, -7.283305058531785e-07]
FORWARD
[5.3 3.7 1.5 0.2]
[-1.49271312  0.39812058  6.62567908]
[0.18351485 0.59823603 0.99867588]
[-1.12074563  0.42781703]
[0.245873   0.60535227]
[1.0300496]
[0.73692551]
BACKWARD
delta
[0.14286502]
activation
[0.245873   0.60535227]
[0.03512665]
nabla_w[-1]
[0.035126652497875666, 0.08648366700956724]
delta
[-0.00928747  0.00663887]
activation
[0.18351485 0.59823603 0.99867588]
activation trans
[0.18351485 0.59823603 0.99867588]
nabla_w[-l]
[0.0012183311458715583, 0.0039716108805738105, 0.006630078778841193]
delta
[ 3.98738082e-04 -3.45018696e-03  2.54048842e-05]
activation
[5.3 3.7 1.5 0.2]
activation trans
[5.3 3.7 1.5 0.2]
nabla_w[-l]
[0.00013464588637683241, 9.399807162156225e-05